AICU Lane following with stable baselines3



Check the recieved GPU (necessary minimum of 16gb GPU RAM)

In [1]:
!nvidia-smi

Fri Dec 11 16:31:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Installation section:
 - Stable - baselines
 - Optuna
 - swig cmake
 - GYM (OpenAI)
 - pip (latest version)
 - GYM Duckietown
 - Virtual display librarys:
  - pyvirtualdisplay
  - piglet
  - xvfb
  - python-opengl





In [2]:
#Install stable baselines
!pip install stable-baselines3[extra] box2d box2d-kengz
!pip install stable-baselines[mpi]==2.10.0

#Install Optuna (hyperparam. optimizer tool)
!pip install optuna

#Install swig, cmake
!apt install swig cmake

#Clone and install OPENAI - GYM
!git clone https://github.com/openai/gym.git
%cd gym/
!pip3 install -e .

#Install and upgrade pip
!python3 -m pip install --upgrade pip
%cd ..

#Clone and install Duckietown-GYM
!git clone https://github.com/duckietown/gym-duckietown.git
%cd gym-duckietown/
!git checkout dcf8dd3
!pip install -e .

#Create virtual monitor
%cd /
!pip install pyvirtualdisplay
!pip install piglet
!apt install xvfb -y
!apt-get install python-opengl -y
!apt-get install ffmpeg freeglut3-dev xvfb 

#Start display
print("\n Starting virtual display... \n")
from pyvirtualdisplay import Display
display = Display(visible=0, size=(640, 480))
display.start()

#Install duckietown world for visualization
#!git clone https://github.com/duckietown/duckietown-world.git
#!git chekcout e069378
#%cd duckietown-world/
#!pip install -r requirements.txt





     |████████████████████████████████| 153kB 12.8MB/s 
     |████████████████████████████████| 1.3MB 24.6MB/s 
     |████████████████████████████████| 430kB 54.8MB/s 
  ERROR: Failed building wheel for box2d-kengz
  Running setup.py clean for box2d-kengz
Failed to build box2d-kengz
    Running setup.py install for box2d-kengz ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-ly0ec0tq/box2d-kengz/setup.py'"'"'; __file__='"'"'/tmp/pip-install-ly0ec0tq/box2d-kengz/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-feoz78pt/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.
     |████████████████████████████████| 256kB 22.5MB/s 
     |████████████████████████████████| 1.4MB 29.4MB/s

In [ ]:
#Chosing the right tensorflow version for compatibility reasons
%tensorflow_version 1.x

Import the necessary librarys:
- gym
- gym envs
- duckietown envs
- display
- os
- numpy
- plt
- PPO
- CnnPolicy
- BaseCallback
- evaluate policy
- OPTUNA
- joblib


In [ ]:
%cd .. 

/


In [ ]:
!dir

bin	 datalab  home	 lib64	opt   run   swift	       tmp    var
boot	 dev	  lib	 media	proc  sbin  sys		       tools
content  etc	  lib32  mnt	root  srv   tensorflow-1.15.2  usr


In [3]:
import os

import numpy as np
import matplotlib.pyplot as plt

%cd content
%cd gym
from gym import envs
import gym

%cd ..
%cd gym-duckietown/
import gym_duckietown.envs.duckietown_env
env1 = 'Duckietown-small_loop-v0'   #Small loop map
env2 = 'Duckietown-udem1-v0'        #More complex urbun environment
env3 = 'Duckietown-straight_road-v0'#Straight road map

from stable_baselines3 import PPO, A2C #Algorithm and policy import 
from stable_baselines3.ppo.policies import CnnPolicy
from stable_baselines3.a2c.policies import CnnPolicy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.evaluation import evaluate_policy

from pyvirtualdisplay import Display
display = Display(visible=0, size=(640, 480))
display.start()

import optuna
import joblib as joblib

#For data visualization
from google.colab import files

#In case you want to save best models automaticly
#!pip install pickle-mixin
#import pickle

/content
/content/gym


INFO:gym-duckietown:gym-duckietown 2018.10.1

INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0


/content
/content/gym-duckietown


Start the basic training with Stable baselines algorithm

In [ ]:
#Create model
model = PPO(CnnPolicy, env1, verbose=0, n_steps=512, batch_size=128, gamma=0.99, 
            gae_lambda=0.9, n_epochs=20, ent_coef=0.0, sde_sample_freq=4, 
           max_grad_norm=0.5, vf_coef=0.5, learning_rate=0.00003, use_sde=True, clip_range= 0.4)

DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:loading map file "/content/gym-duckietown/gym_duckietown/maps/small_loop.yaml"
INFO:gym-duckietown:Starting at [1.105964   0.         0.10533828] 0.7286281210098
INFO:gym-duckietown:Pos: [1.105964   0.         0.10533828] angle 0.7286281210098
INFO:gym-duckietown:using DuckietownEnv
/content/gym/gym/logger.py:30: UserWarning:

WARN: Box bound precision lowered by casting to float32



In [ ]:
#OPTIONAL
#Callback function for debugging, and superviseing


#class SaveOnBestTrainingRewardCallback(BaseCallback):
    
    
# #   Callback for saving a model (the check is done every ``check_freq`` steps)
# #   based on the training reward (in practice, we recommend using ``EvalCallback``).

# #   :param check_freq: (int)
# #   :param log_dir: (str) Path to the folder where the model will be saved.
# #     It must contains the file created by the ``Monitor`` wrapper.
# #   :param verbose: (int)
    
#    def __init__(self, check_freq: int, log_dir: str, verbose=1):
#        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
#        self.check_freq = check_freq
#        self.log_dir = log_dir
#        self.save_path = os.path.join(log_dir, 'best_model')
#        self.best_mean_reward = -np.inf

#    def _init_callback(self) -> None:
#        # Create folder if needed
#        if self.save_path is not None:
#            os.makedirs(self.save_path, exist_ok=True)

#    def _on_step(self) -> bool:
#        if self.n_calls % self.check_freq == 0:

#          # Retrieve training reward
#          x, y = ts2xy(load_results(self.log_dir), 'timesteps')
#          if len(x) > 0:
#              # Mean training reward over the last 100 episodes
#              mean_reward = np.mean(y[-100:])
#              if self.verbose > 0:
#                print(f"Num timesteps: {self.num_timesteps}")
#                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

#              # New best model, you could save the agent here
#              if mean_reward > self.best_mean_reward:
#                  self.best_mean_reward = mean_reward
#                  # Example for saving best model
#                  if self.verbose > 0:
#                    print(f"Saving new best model to {self.save_path}.zip")
#                  self.model.save(self.save_path)

#        return True


In [ ]:
#OPTIONAL
##Create callback and check every 10 steps
#callback = SaveOnBestTrainingRewardCallback(check_freq=10, log_dir='/log_dir')

In [ ]:
#Evaluate pretrained model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10) 

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
#Train agent
#model.learn(total_timesteps=int(100), callback=callback) #With optional Callback function
model.learn(total_timesteps=int(100))

INFO:gym-duckietown:Starting at [0.8959695  0.         1.32563711] 0.9788245454315734
INFO:gym-duckietown:Pos: [0.8959695  0.         1.32563711] angle 0.9788245454315734
INFO:gym-duckietown:Pos: [0.91828944 0.         1.29244343] angle 0.9788245454315734
INFO:gym-duckietown:Pos: [0.94060937 0.         1.25924975] angle 0.9788245454315734
INFO:gym-duckietown:Pos: [0.96292931 0.         1.22605607] angle 0.9788245454315734
INFO:gym-duckietown:Pos: [0.97847054 0.         1.20236288] angle 1.0016934133994646
DEBUG:gym-duckietown:[1.01403639 0.         1.14676549] corresponds to tile at (1, 1) which is not drivable: {'coords': (1, 1), 'kind': 'asphalt', 'angle': 0, 'drivable': False, 'texture': <gym_duckietown.graphics.Texture object at 0x7f6360f2e780>, 'color': array([0.92885894, 0.8702542 , 0.83738281])}
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [0.9655375  0.         1.22258011]
DEBUG

In [ ]:
#Evaluate trained model
environment = model.get_env()
mean_reward, std_reward = evaluate_policy(model, environment, n_eval_episodes=10)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

A streamkimeneten csak az utolsó 5000 sor látható.
INFO:gym-duckietown:Pos: [0.48582464 0.         0.25068168] angle 2.826287325456308
INFO:gym-duckietown:Pos: [0.48617966 0.         0.25079748] angle 2.8263096947729345
INFO:gym-duckietown:Pos: [0.48649787 0.         0.25090126] angle 2.8263577553565806
INFO:gym-duckietown:Pos: [0.486758   0.         0.25098608] angle 2.8263792549235722
INFO:gym-duckietown:Pos: [0.48703352 0.         0.25107593] angle 2.8263796796971303
INFO:gym-duckietown:Pos: [0.48733773 0.         0.25117512] angle 2.826438455886762
INFO:gym-duckietown:Pos: [0.48761534 0.         0.25126561] angle 2.826503753547202
INFO:gym-duckietown:Pos: [0.48792117 0.         0.25136529] angle 2.8265397469275193
INFO:gym-duckietown:Pos: [0.48826384 0.         0.25147697] angle 2.8265410182869646
INFO:gym-duckietown:Pos: [0.48854728 0.         0.25156934] angle 2.8265970420365742
INFO:gym-duckietown:Pos: [0.48882055 0.         0.25165837] angle 2.826647427884076
INFO:gym-duckietow

mean_reward:-699.86 +/- 810.77


Hyperparam tuning with optuna

In [ ]:
#PPO optimizer function

def optimize_ppo(trial):
  #Adjust hyperparams 
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_int('n_steps', 32, 512)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
        'n_epochs': trial.suggest_int('n_epochs',3,5)
    }

#A2C optimizer fucntion

def optimize_a2c(trial):
  #Adjust hyperparams 
    """ Learning hyperparamters we want to optimise"""
    return {
        'n_steps': int(trial.suggest_int('n_steps', 32, 512)),
        'gamma': trial.suggest_loguniform('gamma', 0.9, 0.9999),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-3),
        'ent_coef': trial.suggest_loguniform('ent_coef', 1e-8, 1e-1),
    }


In [ ]:
#Agent optimizer function for PPO

def optimize_agent(trial):
    """ Train the model and optimise
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    model_params = optimize_ppo(trial)
    env = gym.make('Duckietown-straight_road-v0')
    #env = gym.make('Duckietown-small_loop-v0')
    #For multiprocessing
    #env = SubprocVecEnv([lambda: gym.make('Duckietown-small_loop-v0') for i in range(n_cpu)])
    model = PPO(CnnPolicy, env1, verbose=0, **model_params)
    model.learn(100)

    rewards = []
    n_episodes, reward_sum = 0, 0.0

    obs = env.reset()
    while n_episodes < 100:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward

        if done:
            rewards.append(reward_sum)
            reward_sum = 0.0
            n_episodes += 1
            obs = env.reset()

    last_reward = np.mean(rewards)
    trial.report(-1 * last_reward, n_episodes)

    #Handle pruning based on the intermediate value.
    if trial.should_prune():
      raise optuna.TrialPruned()

    return -1 * last_reward

#Agent optimizer function for A2C

def optimize_agent(trial):
    """ Train the model and optimise
        Optuna maximises the negative log likelihood, so we
        need to negate the reward here
    """
    model_params = optimize_a2c(trial)
    env = gym.make('Duckietown-straight_road-v0')
    #env = gym.make('Duckietown-small_loop-v0')
    #For multiprocessing
    #env = SubprocVecEnv([lambda: gym.make('Duckietown-small_loop-v0') for i in range(n_cpu)])
    model = A2C(CnnPolicy, env1, verbose=0, **model_params)
    model.learn(100)

    rewards = []
    n_episodes, reward_sum = 0, 0.0

    obs = env.reset()
    while n_episodes < 100:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward

        if done:
            rewards.append(reward_sum)
            reward_sum = 0.0
            n_episodes += 1
            obs = env.reset()

    last_reward = np.mean(rewards)
    trial.report(-1 * last_reward, n_episodes)

    #Handle pruning based on the intermediate value.
    if trial.should_prune():
      raise optuna.TrialPruned()

    return -1 * last_reward


In [ ]:
#Setting upt the hyperparameter tuner study with pruner 
study = optuna.create_study(pruner=optuna.pruners.PercentilePruner(percentile=70,n_startup_trials=20,n_warmup_steps=25,interval_steps=1),sampler=sampler,)

#Start optimization (Also handling NaN cases)
study.optimize(optimize_agent, n_trials=20, gc_after_trial=True,catch=(float('nan'),))
#study.optimize(optimize_agent, n_trials=1000, n_jobs=4) #
study.best_params

A streamkimeneten csak az utolsó 5000 sor látható.
INFO:gym-duckietown:Pos: [1.12959932 0.         1.43003967] angle 0.8753403092800444
INFO:gym-duckietown:Pos: [1.15522875 0.         1.3993292 ] angle 0.8753403092800444
INFO:gym-duckietown:Pos: [1.18085817 0.         1.36861874] angle 0.8753403092800444
INFO:gym-duckietown:Pos: [1.18325308 0.         1.36582413] angle 0.8491964652851218
INFO:gym-duckietown:Pos: [1.20967654 0.         1.33579414] angle 0.8491964652851218
INFO:gym-duckietown:Pos: [1.20989726 0.         1.33554643] angle 0.8367239830210382
INFO:gym-duckietown:Pos: [1.21767618 0.         1.32651045] angle 0.883311450992154
INFO:gym-duckietown:Pos: [1.21470328 0.         1.33005467] angle 0.8623680240895483
INFO:gym-duckietown:Pos: [1.19750019 0.         1.34966998] angle 0.8392744523033542
INFO:gym-duckietown:Pos: [1.17078007 0.         1.37943633] angle 0.8392744523033542
INFO:gym-duckietown:Pos: [1.19124111 0.         1.35682814] angle 0.8311465314930447
INFO:gym-duckie

{'ent_coef': 4.8521275396586146e-08,
 'gamma': 0.9877401830478091,
 'learning_rate': 2.5039481560708257e-05,
 'n_steps': 255.8067518129065}

In [ ]:
#Extracting the params
BestParams = study.best_params
print(BestParams)
b_n_steps = BestParams["n_steps"]
b_gamma = BestParams["gamma"]
b_learning_rate = BestParams["learning_rate"]
b_ent_coef = BestParams["ent_coef"]
b_n_epochs = BestParams["n_epochs"]

#Creating a model with the calculated hyperparams
model = PPO(CnnPolicy, env1, verbose=0, n_epochs=b_n_epochs , ent_coef=b_ent_coef, gamma=b_gamma, learning_rate=b_learning_rate, n_steps=b_n_steps)
#modela2c = A2C(CnnPolicy, env1, verbose=0, ent_coef=b_ent_coef, gamma=b_gamma, learning_rate=b_learning_rate, n_steps=b_n_steps)


{'n_steps': 255.8067518129065, 'gamma': 0.9877401830478091, 'learning_rate': 2.5039481560708257e-05, 'ent_coef': 4.8521275396586146e-08}


In [ ]:
#Train agent
#model.learn(total_timesteps=int(100), callback=callback) #With optional Callback function
model.learn(total_timesteps=int(100))

In [ ]:
#Evaluate trained model with hyperparameters
mean_reward, std_reward = evaluate_policy(model, env1, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Resume Hyperparameter tuning. To be able to optimize better we need more computing capacity so we save and load the optimized models timely (in case of a crash the models were saved this way)





In [ ]:
for i in range(2):
  #study.optimize(optimize_agent, n_trials=10, gc_after_trial=True, catch=(float('nan'),))
  study.optimize(optimize_agent, n_trials=10, gc_after_trial=True,catch=(float('NaN'),))
  study.best_params
  joblib.dump(study, 'PPOstraightV1.pkl')
  study = joblib.load('PPOstraightV1.pkl')
  print(i)

In [ ]:
#Printing the best trial and its values
study = joblib.load('/content/PPOstraightV1.pkl')
print('Best trial until now:')
print(' Value: ', study.best_trial.value)
print(' Params: ')
for key, value in study.best_trial.params.items():
    print(f'    {key}: {value}')

In [ ]:
#Save tunings into variables
study1 = joblib.load('/content/PPOstraightV1.pkl')
study2 = joblib.load('/content/studyStraightv2seed3a2c.pkl')

Visualize the tuning results

In [ ]:
#Creates a list of the hyperparameter importances
optuna.importance.get_param_importances(study1)

In [ ]:
#Empirical distribution function, cumulative probability
#Demonstrates that what is the possibility of given Objective values ranges
#Study1 - PPO, Study2 - A2C
optuna.visualization.plot_edf([study1, study2])

In [ ]:
#Connects the hyperparam values of each trial and shows their objective values
optuna.visualization.plot_parallel_coordinate(study1)

In [ ]:
#Shows the each hyperparameters cluster
optuna.visualization.plot_slice(study1)